# Statistical Analysis

As we covered ways to discover insights with our data in our previous lesson, a different type of analysis to cover is the use of statistics to further enhance our findings. 

In any sort of business, statistical analysis can help the organization make better decisions based on data, employ correct analysis tactics, make accurate predictions, and so much more. Python can be utilized in ways to get these insights with the use of its functions.

## Single Variable Statistics

We will start off with an introduction to the basics of statistics: `mean`, `median` and `mode`.


 1. `Mean`: The average of the dataset. Best used when the distribution is even and the variance is low
 2. `Median`: The middle of the dataset. Best used as a measure of location to mitigate the effect of outliers in the data
 3. `Mode`: The most common number of the dataset. Used to figure out which number is the most prevailant or where the data is most concentrated in. Sometimes, mode will not exist in the data

We can use pandas and use statistical functions to find these insights:

In [ ]:
import pandas as pd
import numpy as np

df_grades = pd.read_csv('Student Grades.csv')
df_grades

In [ ]:
df_grades['Tuition'].mean()

In [ ]:
df_grades['OfficeHoursParticipated'].median()

In [ ]:
df_grades['ClassesSkipped'].mode()

In [ ]:
# You can also select multiple columns at a time
df_grades[['Tuition','ClassesSkipped']].mean()

One of the key concepts in statistics is the idea of `variance` and `standard deviation`. 

1. `Variance` is a statistical measurement of how spread out the numbers are in a dataset
2. `Standard deviation` is a statistical measurement of how spread out the numbers are, but relative to the mean and is calculated as the square root of the variance

When we know how spread out the data is, we can identify insights such as how reliable our predictions will be due to volitility or identify outliers that affect our data.

Keep in mind that pandas will automatically detect values that are numbers to calculate variance and standard deviation and exclude fields that contain letters. However, we must be mindful of columns that are numbers that do not make sense to have variance and standard deviation as they are categorical. For example, `StudentID` have numbers but they are used to classify a student.

With that being said, we can use the `var()` and `std()` function to find the variance and standard deviation of our DataFrame.

In [ ]:
# Variance of entire DataFrame
df_grades.var(numeric_only=True)

In [ ]:
# Standard deviation of entire DataFrame
df_grades.std(numeric_only=True)

In [ ]:
# Variance of specific column
df_grades['Tuition'].var()

In [ ]:
# Standard deviation of specific column
df_grades['Tuition'].std()

We can also see a snapshot of all the column statistics with the `describe()` function as well:


In [ ]:
df_grades.describe()

## Z-score

A `z-score` is a measurement that describes the distance a value is from the mean of a normal data distribution. `Z-score` is measured in terms of `standard deviations`. A `z-score` typically tells us how far a value is in standard deviations, so that we can understand it in a scaled value.

A `z-score` can be calculated using the following formula: z = (X – μ) / σ


X is the single input value

μ is the population mean

σ is the population standard deviation


With this, we can calculate the standard deviation of each student's participation in office hours to see who spends the most time.

## Identifying Outliers

`Outliers` are defined as data points that are abnormally far from what is considered "normal". Sometimes, outliers causes hinderance to our data, as they might be there due to errors and it is up to us to decide if these abnormal data are to be included. Before we start identifying outliers, we are going to add a new column to our DataFrame `GPA`, based on the `GradeAverage` of the students. We will create this using the `np.select` function.


In [ ]:
# np.select ([list of categories], [list of categories to be replaced, in the same order])
df_grades['GPA'] = np.select([df_grades['GradeAverage'] == 'A', 
                              df_grades['GradeAverage'] == 'B', 
                              df_grades['GradeAverage'] == 'C',
                              df_grades['GradeAverage'] == 'D',
                              df_grades['GradeAverage'] == 'F'],
                             [4,3,2,1,0])

df_grades                     

Data that is far outside the `interquartile range (IQR)` is identified as an outlier, also known as the`IQR rule`. The `IQR` is a measure of how 50% of the "middle" of the data is spread out.

The `IQR` is calculated by subtracting the 75th percentile data point (third quartile) with the 25th percentile data point (first quartile). 

The `IQR rule` is calculated by multipying the `IQR` by 1.5 and adding it to the third quartile and subtracting it from the first quartile. If a number is greater than the third quartile or less than the first quartile after the addition/subtraction, then it can be seen as an outlier.

 Example: 

 * IQR = 20 
 
 * Q3 = 70
 
 * Q1 = 50
 
 * IQR x 1.5 = 30
 
 * Q3 + 30 = 100 --> Anything above 100 is considered an outlier for this dataset 
 
 * Q1 - 30 = 20 --> Anything below 20 is considered an outlier for this dataset
 
 
With this logic, we can identify if any students have a GPA that is an outlier and add this to our data.

In [ ]:
# np.percentile finds the specified percentile of an array
q3 = np.percentile(df_grades['GPA'], 75)
q1 = np.percentile(df_grades['GPA'], 25)
IQR = q3 - q1
IQR

In [ ]:
# Define IQR rule for the GPA column
IQR_rule = (1.5*IQR + q3 < df_grades['GPA']) | (q1 - 1.5*IQR > df_grades['GPA'])

# use np.where to create 'Outlier' column, is a student's GPA an outlier or not
df_grades['Outlier'] = np.where(IQR_rule, 'yes', 'no')
df_grades

## Correlation

So far, we have covered statistics that summarize the aspects of a single column of data. Now, we will explore how to use `correlation` to see how two columns of data will interact with one another.

`Correlation` is the method to evaluate the strength of a relationship between two continous variables. This not only shows the direction between two variables, but also the magnitude of the relationship. `Correlation` values are between -1 and 1, with -1 being the highest value indirectly proportional, and 1 being the highest value directly proportional.

To get an overall snapshot of `correlation` in our DataFrame, we can use the `corr()` function.


In [ ]:
df_grades.corr()

We see that based on the `correlation` function, there is a strong correlation negatively between `ClassesSkipped` and `GPA`, whereas there is a strong positive correlation between `OfficeHoursParticipated` and `GPA`. This makes sense as student skip more classes, the more likely their grades will drop, and the more office hours are attended, the more likely their grades will rise due to more support.

However, as more data is added into the DataFrame, it may be hard to interpret the data by looking at just numbers. The best way to see a correlation matrix is with the use of a heatmap.

Below is a simple heatmap created by `Seaborn`, which is a visualization package that we will cover in a later chapter. With the colors, we are more easily able to identify strong correlations without having to scan individual numbers in the matrix.

In [ ]:
import seaborn as sns
sns.heatmap(df_grades.corr(), cmap='coolwarm')